In [7]:
using Plots, ForwardDiff, Interact, Printf, LinearAlgebra

In [11]:
# check if matrix is symmetric
function isSymmetric(M)
    (m,n) = size(M)   
    if m != n
        return false
    end
    for i in 1:m
        for j in 1:n
            if M[i,j] != M[j,i]
                return false
            end
        end
    end
    return true
end

# one iterate of cholesky factorization
function CholeskyIterate(M) 
    (n,~) = size(M)
    
    ϵ = .1;
    tol = 1e-4
    
    R = similar(M)
    A = similar(M)
    log_PD = true

    # Compute Schur complement matrix
    
    if M[1,1] <= tol
        log_PD = false
        return nothing 
    else
        α = √(M[1,1])
        w = M[1, 2:end]
        R = [α w'/α; zeros(n-1, 1) I(n-1)]
        A = [ 1 zeros(1, n-1); zeros(n-1,1) M[2:end,2:end] - w*w' / (α^2)]
    end
    
    return log_PD, R, A

end


function CholeskyFull(M)
    tol = 1e-4
    
    if isSymmetric(M)
        (n,~) = size(M) 
    
        # Cholesky iterate
        
        Rt = I(n)
        for i in 1:n-1
            (log_PD, R, M) = CholeskyIterate(M[i:end, i:end])
            if log_PD
                 Rt = [I(i-1) zeros(i-1, n-i+1); zeros(n-i+1, i-1) R] * Rt
            else
                @printf("the matrix is not positive definite\n")
                return nothing
            end
        
        end
        if M[2,2] <= tol        
            @printf("Matrix is not PD")
            return nothing
        else
            Rt[n,n] = Rt[n,n] * √(M[2,2])
        end
        return Rt
                  
    else
        @printf("Matrix is not symmetric\n")
        return nothing
    end       
end

CholeskyFull (generic function with 1 method)

In [20]:
x1 = randn(3)
x2 = randn(3)
x3 = randn(3)

X = x1*x1' + x2*x2';
@printf("Cholesky:\n")
K1 = CholeskyFull(X)


@show(norm(K1'K1 - X));


Cholesky:
Matrix is not PD

MethodError: MethodError: no method matching adjoint(::Nothing)
Closest candidates are:
  adjoint(!Matched::Missing) at missing.jl:100
  adjoint(!Matched::Number) at number.jl:193
  adjoint(!Matched::Adjoint{#s162,#s161} where #s161<:Union{StaticArrays.StaticArray{Tuple{N},T,1} where T where N, StaticArrays.StaticArray{Tuple{N,M},T,2} where T where M where N} where #s162) at /Users/babhru/.julia/packages/StaticArrays/1g9bq/src/linalg.jl:72
  ...

In [16]:
K1

3×3 SparseArrays.SparseMatrixCSC{Float64,Int64} with 6 stored entries:
  [1, 1]  =  1.86888
  [1, 2]  =  0.700526
  [2, 2]  =  1.22133
  [1, 3]  =  -0.540022
  [2, 3]  =  -0.684025
  [3, 3]  =  0.10671

In [4]:
# modified Cholesky
function ModifiedCholeskyFull(M)
    ϵ = .1
    tol = 1e-4;
    
    if isSymmetric(M)
        (n,~) = size(M) 

        Rt = I(n)
        for i in 1:n-1
            (log_PD, R, M) = CholeskyIterate(M[i:end,i:end])
            if log_PD
                Rt = [I(i-1) zeros(i-1, n-i+1); zeros(n-i+1, i-1 ) R] * Rt
            else
                @printf("Using moified Cholesky step\n")
                Rt = [I(i-1) zeros(i-1, n-i+1); zeros(n-i+1, i-1 ) R] * Rt
            end                
        end   
        
        if abs(M[2,2]) <= tol
            @printf("Using moified Cholesky step\n")
            Rt[n,n] = Rt[n,n] * ϵ
        else
            Rt[n,n] = Rt[n,n] * √(M[2,2])
        end
        
        return Rt
            
    else
        @printf("Matrix is not symmetric\n")
        return nothing
    end       
end

ModifiedCholeskyFull (generic function with 1 method)

In [5]:
x1 = randn(3)
x2 = randn(3)
x3 = randn(3)

X = x1*x1' + x2*x2';
@printf("Cholesky:\n")
K1 = ModifiedCholeskyFull(X)

@show(norm(K1'K1 - X));


Cholesky:
Using moified Cholesky step
norm(K1' * K1 - X) = 0.010000000000000342
